In [7]:
import enoki as ek
import mitsuba
mitsuba.set_variant('gpu_autodiff_rgb')

from mitsuba.core import Thread, Float
from mitsuba.core.xml import load_file
from mitsuba.python.util import traverse
from mitsuba.python.autodiff import render, write_bitmap

# Load the Cornell Box
Thread.thread().file_resolver().append('cbox/')

# scene = load_file('cbox/cbox_path.xml')
# scene = load_file('cbox/cbox_direct.xml')
# scene = load_file('cbox/cbox_path_tc.xml')
# scene = load_file('cbox/cbox_direct_tc.xml')
scene = load_file('cbox/cbox_path_nee.xml')

# Find differentiable scene parameters
params = traverse(scene)

wall_res = params['wall_bsdf.reflectance.resolution']
wall_data = Float(params['wall_bsdf.reflectance.data'])
print(wall_res, wall_data)

params.keep(['wall_bsdf.reflectance.data'])

2021-11-30 14:08:07 INFO main [xml.cpp:1221] Loading XML file "cbox\cbox_path_nee.xml" ..

2021-11-30 14:08:07 INFO main [xml.cpp:1222] Using variant "gpu_autodiff_rgb"

2021-11-30 14:08:07 INFO main [xml.cpp:355] "cbox\cbox_path_nee.xml": in-memory version upgrade (v2.0.0 -> v2.2.1) ..

2021-11-30 14:08:07 WARN main [PathTextureSpaceIntegrator] PathTextureSpaceIntegrator using shape: wall_shape

2021-11-30 14:08:07 WARN main [PathTextureSpaceIntegrator] PathTextureSpaceIntegrator using integrator:

2021-11-30 14:08:07 INFO main [Scene] Building scene in OptiX ..

[16, 16] [0, 0, 0, 0, 0, .. 758 skipped .., 0.520996, 0.799103, 0.577581, 0.520996, 0.799103]


In [8]:
image_ref = render(scene, spp=256)
crop_size = scene.sensors()[0].film().crop_size()
write_bitmap('out/out_ref.png', image_ref, crop_size)

![out_ref.png](out/out_ref.png)

In [4]:
params['wall_bsdf.reflectance.data'] = ek.full(Float, 1.0, len(wall_data))
params.update()

from mitsuba.python.autodiff import Adam
opt = Adam(params, lr=0.1)

image = render(scene, spp=8)
crop_size = scene.sensors()[0].film().crop_size()
write_bitmap('out/out_start.png', image, crop_size)
write_bitmap('out/wall_start.png', params['wall_bsdf.reflectance.data'],
                 (wall_res[0], wall_res[1]))

for it in range(10):
    # Perform a differentiable rendering of the scene
    image = render(scene, optimizer=opt, unbiased=True, spp=1)

    # Objective: MSE between 'image' and 'image_ref'
    ob_val = ek.hsum(ek.sqr(image - image_ref)) / len(image)

    # Back-propagate errors to input parameters
    ek.backward(ob_val)

    # Optimizer: take a gradient step
    opt.step()

    err_ref = ek.hsum(ek.sqr(wall_data - params['wall_bsdf.reflectance.data']))

    print('%g,%g' % (ob_val[0], err_ref[0]))

image = render(scene, spp=8)
crop_size = scene.sensors()[0].film().crop_size()
write_bitmap('out/out_end.png', image, crop_size)
write_bitmap('out/wall_end.png', params['wall_bsdf.reflectance.data'],
                 (wall_res[0], wall_res[1]))

nan,nan
nan,nan
nan,nan
nan,nan
nan,nan
nan,nan
nan,nan
nan,nan
nan,nan
nan,nan


![start](out/out_start.png)
![end](out/out_end.png)
![start](out/wall_start.png)
![end](out/wall_end.png)